In [76]:
#import necessary package 
from bs4 import BeautifulSoup
import requests

In [77]:
#picking a random header to avoid the block.
import random
random.seed(10)
headerlist = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
     "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991",
           "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36 OPR/42.0.2393.94",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36 OPR/47.0.2631.39",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
           "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:56.0) Gecko/20100101 Firefox/56.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko"]
user_agent = random.choice(headerlist)
fake_headers = {'User-Agent': user_agent}
#Remark:I find out that different fake agent will create different searrch result.

In [78]:
#downloading the original html
booking_url="https://www.booking.com/searchresults.html"
my_params={"ss":"Taipei","checkin":"2023-11-30","checkout":"2023-11-31","offset":25}
r=requests.get(booking_url , params=my_params,headers=fake_headers)
if r.status_code == requests.codes.ok:
    print("Ok")
    print(r.url)
    soup=BeautifulSoup(r.text,"html.parser")


Ok
https://www.booking.com/searchresults.html?ss=Taipei&checkin=2023-11-30&checkout=2023-11-31&offset=25


In [79]:
import pandas as pd
hotel_info=pd.DataFrame(columns=["name","location","price","rating","distance","comments"])
hotel_info

,name,location,price,rating,distance,comments


In [80]:
#finding hotel name
hotel_name=soup.find_all(class_="f6431b446c a15b38c233")
hotel_name_Series=pd.Series(len(hotel_name))
for i in range(len(hotel_name)):
    hotel_name_Series[i]=hotel_name[i].string
hotel_info["name"]=hotel_name_Series

In [81]:
#finding hotel location
hotel_location=soup.find_all(attrs={"data-testid": "address"},class_="aee5343fdb def9bc142a")
hotel_location_Series=pd.Series(len(hotel_location))
for i in range(len(hotel_location)):
    hotel_location_Series[i]=hotel_location[i].string
hotel_info["location"]=hotel_location_Series

In [82]:
#finding hotel price
hotel_price=soup.find_all(attrs={"data-testid": "price-and-discounted-price"},class_="f6431b446c fbfd7c1165 e84eb96b1f")
hotel_price_Series=pd.Series(len(hotel_price))
for i in range(len(hotel_price)):
    hotel_price_Series[i]=hotel_price[i].string
hotel_info["price"]=hotel_price_Series

In [83]:
#finding hotel score
#remark: some results have no score since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external score.
#treatment: it is not pretty fair to compare the external score with booking.com internal score, so there I treat all the newcomers without internal score as nan.  
from math import nan #in order to assign nana value to the hotel with no internal score
hotel_score=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
hotel_score_Series=pd.Series(len(hotel_score))
print(len(hotel_score))
for i in range(len(hotel_score)):
    sub_soup = BeautifulSoup(str(hotel_score[i]),'html.parser')
    try:
        hotel_score_Series[i]=float(sub_soup.find(class_="a3b8729ab1 d86cee9b25").text)
    except:
        hotel_score_Series[i]= nan
#common class for both types
#with score
#aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
#without score
#aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
hotel_info["rating"]=hotel_score_Series

25


In [84]:
#finding hotel distance
hotel_distance=soup.find_all(attrs={"data-testid": "distance"})
hotel_distance_Series=pd.Series(len(hotel_distance))
for i in range(len(hotel_distance)):
    hotel_distance_Series[i]=hotel_distance[i].string
hotel_info["distance"]=hotel_distance_Series

In [85]:
#finding hotel comments
#remark: some results have no comments since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external comments.
#treatment: it is not pretty fair to compare the external comments with booking.com internal comments, so there I treat all the newcomers without internal comments as nan.  
hotel_comments=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
hotel_comments_Series=pd.Series(len(hotel_comments))
print(len(hotel_comments))
for i in range(len(hotel_comments)):
    sub_soup = BeautifulSoup(str(hotel_comments[i]),'html.parser')
    try:
        hotel_comments_Series[i]=sub_soup.find(class_="a3b8729ab1 e6208ee469 cb2cbb3ccb").text
    except:
        hotel_comments_Series[i]= nan
hotel_info["comments"]=hotel_comments_Series

25


In [86]:
hotel_info

,name,location,price,rating,distance,comments
0,Courtyard by Marriott Taipei,"Nangang District, Taipei","TWD 4,800",8.2,9 km from centre,Very good
1,Beauty Hotels Taipei - B7 Journey,"Zhongzheng District, Taipei","TWD 1,710",7.4,0.6 km from centre,Good
2,Via Loft Hotel,"Zhongshan District, Taipei","TWD 2,080",8.3,1.5 km from centre,Very good
3,喜客商旅 Seeker Hotel - Sanchong,"Sanchong District , Taipei","TWD 1,049",7.5,3.6 km from centre,Good
4,"Hotel Resonance Taipei, Tapestry Collection by...","Zhongzheng District, Taipei","TWD 7,565",9.2,0.6 km from centre,Superb
5,Fun Inn Taipei Hostel,"Zhongzheng District, Taipei",TWD 720,7.5,0.7 km from centre,Good
6,Walker Hotel - Sanchong,"Sanchong District , Taipei",TWD 998,7.2,3 km from centre,Good
7,The Landis Taipei,"Zhongshan District, Taipei","TWD 6,082",8.9,2.2 km from centre,Fabulous
8,Hotel Fun - Linsen,"Zhongshan District, Taipei","TWD 1,877",8.1,1.9 km from centre,Very good
9,Finders Hotel-Fu Qian,"Zhongzheng District, Taipei","TWD 2,250",8.4,0.6 km from centre,Very good


In [19]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import random
import sys
import numpy as np

def booking_crawler(location:str,checkin:str,checkout:str):

    #picking a random header to avoid the block.
    headerlist = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991",
        "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36 OPR/42.0.2393.94",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36 OPR/47.0.2631.39",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
        "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:56.0) Gecko/20100101 Firefox/56.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko"]
    user_agent = random.choice(headerlist)
    fake_headers = {'User-Agent': user_agent}
    #Remark:I find out that different fake agent will create different searrch result.

    page_n=25
    hotel_info=pd.DataFrame(columns=["name","location","price","rating","distance","comments"])
    hotel_info_temp=pd.DataFrame(columns=["name","location","price","rating","distance","comments"])
    offset=0
    while hotel_info.shape[0]<50:
        booking_url="https://www.booking.com/searchresults.html"
        my_params={"ss":location,"checkin":checkin,"checkout":checkout,"offset":offset}
        r=requests.get(booking_url , params=my_params,headers=fake_headers)
        if r.status_code == requests.codes.ok:
            print("Ok")
            soup=BeautifulSoup(r.text,"html.parser")

        #finding hotel name
        hotel_name=soup.find_all(class_="f6431b446c a15b38c233")
        hotel_name_Series=pd.Series(len(hotel_name))
        for i in range(len(hotel_name)):
            hotel_name_Series[i]=hotel_name[i].string
        hotel_info_temp["name"]=hotel_name_Series

        #finding hotel location
        hotel_location=soup.find_all(attrs={"data-testid": "address"},class_="aee5343fdb def9bc142a")
        hotel_location_Series=pd.Series(len(hotel_location))
        for i in range(len(hotel_location)):
            hotel_location_Series[i]=hotel_location[i].string
        hotel_info_temp["location"]=hotel_location_Series

        #finding hotel price
        hotel_price=soup.find_all(attrs={"data-testid": "price-and-discounted-price"},class_="f6431b446c fbfd7c1165 e84eb96b1f")
        hotel_price_Series=pd.Series(len(hotel_price))
        for i in range(len(hotel_price)):
            hotel_price_Series[i]=hotel_price[i].string
        hotel_info_temp["price"]=hotel_price_Series

        #finding hotel score
        #remark: some results have no score since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external score.
        #treatment: it is not pretty fair to compare the external score with booking.com internal score, so there I treat all the newcomers without internal score as nan.  
        from math import nan #in order to assign nana value to the hotel with no internal score
        hotel_score=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
        hotel_score_Series=pd.Series(len(hotel_score))
        for i in range(len(hotel_score)):
            sub_soup = BeautifulSoup(str(hotel_score[i]),'html.parser')
            try:
                hotel_score_Series[i]=sub_soup.find(class_="a3b8729ab1 d86cee9b25").text
            except:
                hotel_score_Series[i]= nan

        #common class for both types
        #with score
        #aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
        #without score
        #aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
        hotel_info_temp["rating"]=hotel_score_Series

        #finding hotel distance
        hotel_distance=soup.find_all(attrs={"data-testid": "distance"})
        hotel_distance_Series=pd.Series(len(hotel_distance))
        for i in range(len(hotel_distance)):
            hotel_distance_Series[i]=hotel_distance[i].string
        hotel_info_temp["distance"]=hotel_distance_Series

        #finding hotel comments
        #remark: some results have no comments since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external comments.
        #treatment: it is not pretty fair to compare the external comments with booking.com internal comments, so there I treat all the newcomers without internal comments as nan.  
        hotel_comments=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
        hotel_comments_Series=pd.Series(len(hotel_comments))
        for i in range(len(hotel_comments)):
            sub_soup = BeautifulSoup(str(hotel_comments[i]),'html.parser')
            try:
                hotel_comments_Series[i]=sub_soup.find(class_="a3b8729ab1 e6208ee469 cb2cbb3ccb").text
            except:
                hotel_comments_Series[i]= np.nan
        hotel_info_temp["comments"]=hotel_comments_Series
        offset+=page_n
        hotel_info=hotel_info.append(hotel_info_temp,ignore_index=True)
        print(hotel_info.shape)
        hotel_info=hotel_info.dropna()
    return hotel_info


In [20]:
hotel=booking_crawler("Paris","2023-11-29","2023-11-30")

Ok


C:\Users\johan\AppData\Local\Temp\ipykernel_17964\4097505180.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hotel_info=hotel_info.append(hotel_info_temp,ignore_index=True)


(26, 6)
Ok
(41, 6)
Ok
(65, 6)


In [21]:
hotel

,name,location,price,rating,distance,comments
0,Hotel Ernest,"10th arr., Paris","TWD 6,915",8.8,2.2 km from centre,Fabulous
1,Atala powered by Sonder,"8th arr., Paris","TWD 7,580",8.1,4.1 km from centre,Very good
2,Le Katorze Hôtel,"17th arr., Paris","TWD 6,665",8.4,4.2 km from centre,Very good
3,Zoku Paris,"17th arr., Paris","TWD 5,295",9.1,5.3 km from centre,Superb
4,MIJE MARAIS,"4th arr., Paris","TWD 4,110",7.5,0.6 km from centre,Good
...,...,...,...,...,...,...
59,La Dépendance,"1st arr., Paris","TWD 7,631",8.9,1.5 km from centre,Fabulous
60,Hôtel Fabric,"11th arr., Paris","TWD 5,700",9.2,1.7 km from centre,Superb
61,Hotel West-End,"8th arr., Paris","TWD 9,697",8.9,3.7 km from centre,Fabulous
62,Appart'City Collection Paris Gare de Lyon,"12th arr., Paris","TWD 6,229",8.9,2.9 km from centre,Fabulous


In [22]:
hotel=hotel.dropna()


In [23]:
hotel

,name,location,price,rating,distance,comments
0,Hotel Ernest,"10th arr., Paris","TWD 6,915",8.8,2.2 km from centre,Fabulous
1,Atala powered by Sonder,"8th arr., Paris","TWD 7,580",8.1,4.1 km from centre,Very good
2,Le Katorze Hôtel,"17th arr., Paris","TWD 6,665",8.4,4.2 km from centre,Very good
3,Zoku Paris,"17th arr., Paris","TWD 5,295",9.1,5.3 km from centre,Superb
4,MIJE MARAIS,"4th arr., Paris","TWD 4,110",7.5,0.6 km from centre,Good
...,...,...,...,...,...,...
59,La Dépendance,"1st arr., Paris","TWD 7,631",8.9,1.5 km from centre,Fabulous
60,Hôtel Fabric,"11th arr., Paris","TWD 5,700",9.2,1.7 km from centre,Superb
61,Hotel West-End,"8th arr., Paris","TWD 9,697",8.9,3.7 km from centre,Fabulous
62,Appart'City Collection Paris Gare de Lyon,"12th arr., Paris","TWD 6,229",8.9,2.9 km from centre,Fabulous
